# GoogLeNet - 更深的卷积神经网络

<img src="http://zh.gluon.ai/_images/googlenet.png" width="800">

GoogLeNet中有多个四个并行卷积层的块。这个块一般叫做Inception，其基于**Network in network**的思想做了很大的改进。我们先看下如何定义一个下图所示的Inception块。

<img src="http://zh.gluon.ai/_images/inception.svg" width="400">




In [1]:
import mxnet as mx
import numpy as np

from mxnet import nd
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import autograd

ctx = mx.gpu()
mx.random.seed(1)
import utils

In [2]:
# 4条path
class Inception(gluon.Block):
    def __init__(self, n1_1, n2_1, n2_3, n3_1, n3_5, n4_1, debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        self.p1_conv_1 = nn.Conv2D(n1_1, kernel_size=1, activation='relu')
        self.p2_conv_1 = nn.Conv2D(n2_1, kernel_size=1, activation='relu')
        self.p2_conv_3 = nn.Conv2D(n2_3, kernel_size=3, padding=1, activation='relu')
        self.p3_conv_1 = nn.Conv2D(n3_1, kernel_size=1, activation='relu')
        self.p3_conv_5 = nn.Conv2D(n3_5, kernel_size=5, padding=2, activation='relu')
        self.p4_pool_3 = nn.MaxPool2D(pool_size=3, padding=1, strides=1)
        self.p4_conv_1 = nn.Conv2D(n4_1, kernel_size=1, activation='relu')
        
    def forward(self, X):
        p1 = self.p1_conv_1(X)
        p2 = self.p2_conv_3(self.p2_conv_1(X))
        p3 = self.p3_conv_5(self.p3_conv_1(X))
        p4 = self.p4_conv_1(self.p4_pool_3(X))
        
        if self.debug:
            print("p1 out shape : ", p1.shape)
            print("p2 out shape : ", p2.shape)
            print("p3 out shape : ", p3.shape)
            print("p4 out shape : ", p4.shape)
        return nd.concat(p1, p2, p3, p4, dim=1)

In [3]:
incp = Inception(64, 96, 128, 16, 32, 32, debug=True)
incp.initialize(ctx=ctx)

X = nd.random.normal(shape=(32, 3, 64, 64), ctx=ctx)
incp(X).shape

p1 out shape :  (32, 64, 64, 64)
p2 out shape :  (32, 128, 64, 64)
p3 out shape :  (32, 32, 64, 64)
p4 out shape :  (32, 32, 64, 64)


(32, 256, 64, 64)

## Inception的思想

* 1.单个$1 \times 1$卷积。
* 2.$1 \times 1$卷积接上$3 \times 3$卷积。通常前者的通道数少于输入通道，这样减少后者的计算量。后者加上了$padding=1$使得输出的长宽和输入一致。
* 3.同2，但换成了$5 \times 5$卷积，因此$padding=2$才能保证输出的长宽和输入一致。
* 4.和1类似，但卷积前用了最大池化层。

## Inception提出的动机

参考博文：[CNN架构优化之二：GoogLeNet Incepetion V1](https://zhuanlan.zhihu.com/p/31809031)

一般而言，提升网络性能最直接的方法就是增加网络的深度和宽度，这也就意味着巨大的参数，这就更容易导致Over-fitting和巨大的计算量。解决上述两个缺点的根本方法是将全连接层甚至一般的卷积层都转化为稀疏连接。主要有两个原因：
* 现实世界中的生物神经系统的连接也是稀疏的；
* 文献《Provable bounds for learning some deep representations》表明，对于大规模稀疏的神经网络，可以通过分析激活值的统计特性和对高度相关的数据进行聚类来逐层构建出一个最优网络。这点表明臃肿的稀疏网络可能可以在不损失性能的前提下被简化;

早些时候，为了打破网络对称性和提高学习能力，传统的网络都使用了随机稀疏连接，但是计算机软硬件对非对称稀疏数据的计算效率很差，所以在AlexNet中又重新启用了全连接层，目的是为了更好的优化并行运算。

所以，现在的问题是有没有一种方法，既能保持网络结构的稀疏性，又能利用密集矩阵的高计算性能?大量文献表明,可以将稀疏矩阵聚类为较为密集的子矩阵来提高计算性能，据此，该论文提出了名为<font color="red">**Inception**</font>的结构来实现这个方法。

Inception结构的主要思路是如何用密集成分来近似最优的局部稀疏结构。因此作者首先提出了下图左的结构，之后为了减少参数，加入了$1 \times 1$的卷积来进行降维，而此思路来自于Network in network。

<img src="https://pic1.zhimg.com/80/v2-ca313c6a87d0ba064372171cc7d1be72_hd.jpg">

对上图的说明如下：

* 采用不同大小的卷积核意味着不同大小视野或者感受野，最后的拼接意味着**不同尺度特征的融合**；

* 之所以采用1、3、5大小的卷积核，主要是为了方便对齐，设定卷积步长stride=1之后，只要分别设定pad=0、1、2，则卷积之后便可以得到相同维度的特征，然后这些特征就可以直接拼接在一起了；

* 很多文献表明Pooling效果很好，所以Inception也嵌入了Pooling；

* 网络越深，特征越抽象，而且每个特征所涉及的感受野更大，因此随着层数的增加，3x3和5x5的卷积核比例也要增加。

* 由于使用5x5的卷积核仍然会带来巨大的计算量，所以，文章借鉴了《Network in Network》中的方法，采用1x1卷积核来进行降维。例如：上一层的输出为100x100x128，经过具有256个输出的5x5卷积层之后(stride=1，pad=2)，输出数据为100x100x256。其中，卷积层的参数为128x5x5x256。假如上一层输出先经过具有32个输出的1x1卷积层，再经过具有256个输出的5x5卷积层，那么最终的输出数据仍为为100x100x256，但卷积参数量已经减少为128x1x1x32+32x5x5x256，大约减少了4倍。

## GoogLeNet

下图为GoogLeNet的详细的网络参数：

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-2.png" width="850">

对该网络的说明如下：

* GoogLeNet采用了模块化的结构，方便添加和修改；

* 网络最后采用了average pooling来代替全连接层，想法来自《Network in Network》，事实证明可以将TOP1 accuracy提高0.6%；

* 虽然移除了全连接层，但是网络中依旧使用可Dropout；

* 为了避免梯度消失，网络额外增加了2个辅助的softmax用于向前传导梯度。文章中说这两个辅助的分类器的loss应该加一个衰减系数，但看caffe中的model也没有加任何衰减。

我们只实现GoogLeNet中的一个输出，并将整个网络分为六个阶段，每个阶段定义一个``gluon.nn.Squential``，最后再把所有的这些块连接起来。

In [4]:
class GoogLeNet(gluon.Block):
    def __init__(self, num_classes, verbose=False, **kwargs):
        super().__init__(**kwargs)
        with self.name_scope():
            self.verbose = verbose
            # first block(same padding)
            b1 = gluon.nn.Sequential()
            b1.add(
                gluon.nn.Conv2D(64, kernel_size=7, strides=2, padding=3, activation='relu'),
                gluon.nn.MaxPool2D(pool_size=3, strides=2)
            )
            # second block
            b2 = gluon.nn.Sequential()
            b2.add(
                gluon.nn.Conv2D(64, kernel_size=1), # 3X3 reduce
                gluon.nn.Conv2D(192, kernel_size=3, strides=1, padding=1, activation='relu'), # 3X3
                gluon.nn.MaxPool2D(pool_size=3, strides=2)
            )
            # Inception 3a (64, 96, 128, 16, 32, 32)
            # Inception 3b (128, 128, 192, 32, 96, 64)
            b3 = gluon.nn.Sequential()
            b3.add(
                Inception(64, 96, 128, 16, 32, 32),
                Inception(128, 128, 192, 32, 96, 64),
                gluon.nn.MaxPool2D(pool_size=3, strides=2)
            )
            # Inception (4a) (192, 96, 208, 16, 48, 64)
            # Inception (4b) (160, 112, 224, 24, 64, 64)
            # Inception (4c) (128, 128, 256, 24, 64, 64)
            # Inception (4d) (112, 144, 288, 32, 64, 64)
            # Inception (4e) (256, 160, 320, 32, 128, 128)
            b4 = gluon.nn.Sequential()
            b4.add(
                Inception(192, 96, 208, 16, 48, 64),
                Inception(160, 112, 224, 24, 64, 64),
                Inception(128, 128, 256, 24, 64, 64),
                Inception(112, 144, 288, 32, 64, 64),
                Inception(256, 160, 320, 32, 128, 128),
                gluon.nn.MaxPool2D(pool_size=3, strides=2)
            )
            # Inception (5a) (256, 160, 320, 32, 128, 128)
            # Inception (5b) (384, 192, 384, 48, 128, 128)
            b5 = gluon.nn.Sequential()
            b5.add(
                Inception(256, 160, 320, 32, 128, 128),
                Inception(384, 192, 384, 48, 128, 128),
                # 这里原论文是7，但是因为我们训练的输入尺寸只有96， 因为我们改为2
                gluon.nn.AvgPool2D(pool_size=2, strides=1)
            )
            b6 = gluon.nn.Sequential()
            b6.add(
                gluon.nn.Dropout(0.4),
                gluon.nn.Flatten(),
                gluon.nn.Dense(num_classes),
            )
            # chain block together
            self.net = gluon.nn.Sequential()
            self.net.add(b1, b2, b3, b4, b5, b6)
        
    def forward(self, X):
        out = X
        for i, blk in enumerate(self.net):
            out = blk(out)
            if self.verbose:
                print("blk%d output : %s" % (i+1, out.shape))
        return out

In [5]:
gln = GoogLeNet(num_classes=10, verbose=True)
gln.initialize(ctx=ctx)

X = nd.random.normal(shape=(32, 3, 96, 96), ctx=ctx)
y = gln(X)

blk1 output : (32, 64, 23, 23)
blk2 output : (32, 192, 11, 11)
blk3 output : (32, 480, 5, 5)
blk4 output : (32, 832, 2, 2)
blk5 output : (32, 1024, 1, 1)
blk6 output : (32, 10)


## 训练

In [6]:
from time import time

batch_size = 64
train_data, test_data = utils.load_dataset(batch_size, resize=96, data_type='cifar10')

gln = GoogLeNet(num_classes=10, verbose=False)
gln.collect_params().initialize(mx.init.Xavier(), ctx=ctx, force_reinit=True)

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(gln.collect_params(), 'sgd', {'learning_rate' : 0.1})

epochs = 20

niter = 0
moving_loss = 0.0
smoothing_constant = 0.9

from time import time
for epoch in range(epochs):
    start = time()
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = gln(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)
        
        niter += 1
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = smoothing_constant * moving_loss + (1-smoothing_constant) * curr_loss
        estimated_loss = moving_loss / (1 - smoothing_constant**niter)
    
    train_acc = utils.evaluate_accuracy_gluon(train_data, gln, ctx)
    test_acc = utils.evaluate_accuracy_gluon(test_data, gln, ctx)
    print("Epoch %d, Moving Train Avg loss %.5f, Train acc %.5f, Test acc %.5f, Time consume %.5f s."
         % (epoch, estimated_loss, train_acc, test_acc, time() - start))

Epoch 0, Moving Train Avg loss 2.03005, Train acc 0.28000, Test acc 0.28670, Time consume 69.89280 s.
Epoch 1, Moving Train Avg loss 1.69580, Train acc 0.41744, Test acc 0.41910, Time consume 68.78216 s.
Epoch 2, Moving Train Avg loss 1.44750, Train acc 0.50410, Test acc 0.49940, Time consume 68.90851 s.
Epoch 3, Moving Train Avg loss 1.15227, Train acc 0.59978, Test acc 0.57780, Time consume 68.84130 s.
Epoch 4, Moving Train Avg loss 1.04971, Train acc 0.64476, Test acc 0.62740, Time consume 68.90965 s.
Epoch 5, Moving Train Avg loss 0.92597, Train acc 0.69340, Test acc 0.66270, Time consume 68.88063 s.
Epoch 6, Moving Train Avg loss 0.85547, Train acc 0.74402, Test acc 0.69830, Time consume 68.98737 s.
Epoch 7, Moving Train Avg loss 0.83491, Train acc 0.78594, Test acc 0.73010, Time consume 68.87968 s.
Epoch 8, Moving Train Avg loss 0.71385, Train acc 0.81424, Test acc 0.74510, Time consume 68.92734 s.
Epoch 9, Moving Train Avg loss 0.61888, Train acc 0.80940, Test acc 0.73530, Time 